## Building a zarr store from Sentinel-1 sigma0 COGs

### Necessary imports

In [1]:
import zarr
import tifffile
import numpy as np
import pystac_client as pc
import xarray as xr
import stackstac
from pyproj import CRS
from dask.distributed import LocalCluster
import os
import dask.array as da
from dask.diagnostics import ProgressBar
from numcodecs import Blosc
import rasterio
import rioxarray
import pandas as pd
import warnings
from datetime import datetime
#os.environ["ZARR_V3_EXPERIMENTAL_API"] = "1"

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
def lookup(arr1, arr2):
    '''
    Get Index of values from arr2 in arr1
    '''
    lookup = {val: idx for idx, val in enumerate(arr1)}
    indices = np.array([lookup.get(val, np.nan) for val in arr2])

    return indices

In [ ]:
def get_idx(array, value):
    return np.where(array==value)[0][0]

In [ ]:
def load_data(item, pol):
    return rioxarray.open_rasterio(item.assets[pol].href).load().expand_dims(time=pd.to_datetime([item.properties["datetime"]]).tz_convert(None))

Start Local Cluster Client

In [ ]:
#client.shutdown()

In [ ]:
client = LocalCluster().get_client()
client.dashboard_link

### STAC Client

Get Data from STAC, query by Tile and Month

In [ ]:
pc_client = pc.Client.open("https://stac.eodc.eu/api/v1")
time_range = "2024-01-01/2024-02-01"
bbox_aut = [16, 46, 18, 49]

In [ ]:
search = pc_client.search(
    collections=["SENTINEL1_SIG0_20M"],
    datetime=time_range,
    #bbox = bbox_aut
    query={"Equi7_TileID": {"eq": "EU020M_E051N015T3"}}
)

In [ ]:
items_eodc = search.item_collection()
items_eodc

In [ ]:
item_list = list(items_eodc)[::-1]

In [ ]:
grouped_items = [[]]
i=0
for item in item_list:
    
    if not grouped_items[i]:
        grouped_items[i].append(item)
    
    else: 
        if get_datetime(item) - get_datetime(grouped_items[i][-1]) <= pd.Timedelta(seconds=50):
            grouped_items[i].append(item)

        else:
            grouped_items.append([item])
            i+=1

Load Data with StackStac, boundary box has to be set according to data or problems occur (which is why only load tile by tile)

In [ ]:
def get_datetime(item):
    return datetime.strptime(item.properties["datetime"], "%Y-%m-%dT%H:%M:%SZ")

In [ ]:
get_datetime(item_list[1]) - get_datetime(item_list[0])

In [ ]:
timestamp_str = item_id.split('_')[1]  # '20240201T165055'

# Parse it into a datetime object
dt = datetime.strptime(timestamp_str, "%Y%m%dT%H%M%S")

In [ ]:
data=[]
for item in item_list[:6]:
    
    d = load_data(item, "VH")

    if not data:
        data.append(d)

    else:
        if d.time.values-data[-1].time.values <= pd.Timedelta(seconds=50):
            d = xr.where(d.values==-9999, data[-1], d.values, keep_attrs=True)
            data[-1]=d
        else:
            data.append(d)

In [ ]:
data = xr.concat(data, dim="time")
data = data.squeeze()
#data = data.sortby("time")

In [ ]:
data

In [ ]:
# time_dim = 20
# x_dim = 15000
# y_dim = 15000

# time = pd.date_range(start="2024-01-01", periods=time_dim, freq="D")
# x = np.arange(5100000, 5400000, 20)
# y = np.arange(1800000, 1500000, -20)

# # Create dummy data: fill with zeros (or any int16 value)
# data = np.zeros((time_dim, y_dim, x_dim), dtype=np.int16)

# # Create DataArray
# data = xr.DataArray(
#     data,
#     dims=["time", "y", "x"],
#     coords={"time": time, "x": x, "y": y},
#     name="example"
# )

In [ ]:
# crs = CRS.from_wkt(item_list[0].properties["proj:wkt2"])
# res = item_list[0].properties["gsd"]
# epsg = crs.to_epsg()
# bbox = item_list[0].properties["proj:bbox"]

In [ ]:
# data = stackstac.stack(items_eodc,
#                         epsg=epsg,
#                         assets=["VH"],
#                         bounds = bbox,
#                         resolution=res,
#                         chunksize=[66,1,100,100],
#                         fill_value=-9999,
#                         rescale=False,
#                         snap_bounds=False)


# data = data.squeeze()
# data = data.drop_vars([coord for coord in data.coords if coord not in ["x","y","time"]])
# data.attrs = {}

### Subsets

For testing, split data to smaller subsets

In [ ]:
# time_dim = 2
# x_dim = 15000
# y_dim = 15000

# # Create coordinate values
# time = pd.date_range(start="2024-01-01", periods=time_dim, freq="D")
# x = mapping_x
# y = mapping_y

# # Create dummy data: fill with zeros (or any int16 value)
# data = np.zeros((time_dim, y_dim, x_dim), dtype=np.int16)

# # Create DataArray
# data = xr.DataArray(
#     data,
#     dims=["time", "y", "x"],
#     coords={"time": time, "x": x, "y": y},
#     name="example"
# )


In [ ]:
# time_dim = 2
# x_dim = 15000
# y_dim = 15000

# # Create coordinate values
# time = pd.date_range(start="2024-01-10", periods=time_dim, freq="D")
# x = mapping_x2
# y = mapping_y

# # Create dummy data: fill with zeros (or any int16 value)
# data2 = np.zeros((time_dim, y_dim, x_dim), dtype=np.int16)

# # Create DataArray
# data2 = xr.DataArray(
#     data2,
#     dims=["time", "y", "x"],
#     coords={"time": time, "x": x, "y": y},
#     name="example"
# )

In [ ]:
# data2

### Building zarr store from scratch

#### Initialize empty zarr store

In [ ]:
mapping_x = np.arange(5100010, 5400000, 20)
mapping_y = np.arange(1799990, 1500000, -20)

In [ ]:
shape = (data.time.shape[0],15000,15000)
chunk_shape = (data.time.shape[0],100,100)
shard_shape = (data.time.shape[0],7500,7500)
compressors_array = zarr.codecs.BloscCodec()
x_shape = (15000) #subset["x"].shape
y_shape = (15000) #subset["y"].shape
time_shape = data.time.shape

In [ ]:
store = zarr.storage.LocalStore("empty_2.zarr")
root = zarr.create_group(store=store, overwrite=True)
s1sig0 = root.create_group("s1sig0")

In [ ]:
overwrite=True

vh_array = s1sig0.create_array(name="VH",
                shape=shape,
                shards=shard_shape,
                chunks=chunk_shape,
                compressors=compressors_array,
                dtype="int16",
                fill_value=-9999,
                dimension_names=["time", "x", "y"],
                #attributes={"_FillValue": -9999},
                overwrite=overwrite)

x_array = s1sig0.create_array(name="x",
                shape=x_shape,
                chunks=(15000,),
                dtype="float64",
                dimension_names=["x"],
                attributes={"_FillValue": "AAAAAAAA+H8="}, #fill value is NaN
                overwrite=overwrite)

y_array = s1sig0.create_array(name="y",
                shape=y_shape,
                chunks=(15000,),
                dtype="float64",
                dimension_names=["y"],
                attributes={"_FillValue": "AAAAAAAA+H8="}, #fill value is NaN
                overwrite=overwrite)

time_array = s1sig0.create_array(name="time",
                shape=time_shape,
                chunks=time_shape,
                dtype="int64",
                dimension_names=["time"],
                attributes={"units": "seconds since 2014-10-01 00:00:00",
                            "calendar": "proleptic_gregorian"},
                overwrite=overwrite)

#### Filling zarr store

In [ ]:
origin = np.datetime64("2014-10-01T00:00:00")
times = data.time.values.astype("datetime64[s]")
time_delta = (times - origin).astype("timedelta64[s]").astype("int64")

In [ ]:
x_min, x_max = [get_idx(mapping_x, data["x"].values[0]), get_idx(mapping_x, data["x"].values[-1])+1]
y_min, y_max = [get_idx(mapping_y, data["y"].values[0]), get_idx(mapping_y, data["y"].values[-1])+1]

In [ ]:
x_array[:] = data.x.values
y_array[:] = data.y.values
time_array[:] = time_delta

In [ ]:
vh_array[:, :, :] = data.values

In [ ]:
zarr.consolidate_metadata(store)
zarr.consolidate_metadata(store, path="s1sig0")

#### Appending to existing store

In [ ]:
store = zarr.storage.LocalStore("empty_2.zarr")
group = zarr.group(store=store)["s1sig0"]

In [ ]:
origin = np.datetime64("2014-10-01T00:00:00")
times = data["time"].values.astype("datetime64[s]")
time_delta = (times - origin).astype("timedelta64[s]").astype("int64")

In [ ]:
existing_x = group["x"][:]
existing_y = group["y"][:]
existing_times = group["time"][:]

In [ ]:
new_x = np.setdiff1d(data.x, existing_x)
new_y = np.setdiff1d(data.y, existing_y)
new_times = np.setdiff1d(time_delta, existing_times)

new_shape_x = np.append(existing_x, new_x).shape[0]
new_shape_y = np.append(existing_y, new_y).shape[0]
new_shape_time = np.append(existing_times, new_times).shape[0]

In [ ]:
group["x"].append(new_x)
group["y"].append(new_y)
group["time"].append(new_times)

In [ ]:
indices_x = lookup(group["x"][:], data.x.values)
indices_y = lookup(group["y"][:], data.y.values)
indices_times = lookup(group["time"][:], time_delta)

In [ ]:
group["VH"].resize((new_shape_time,new_shape_x,new_shape_y))

In [ ]:
for idx, timestamp in zip(indices_times, data.time):
    group["VH"][idx, indices_x[0]:indices_x[-1], indices_y[0]:indices_y[-1]] = data.sel(time = timestamp).values

In [ ]:
zarr.consolidate_metadata(store)
zarr.consolidate_metadata(store, path="s1sig0")

### Building zarr store with xarray

In [ ]:
subset.to_dataset(name="VH").to_zarr(store="small_example.zarr", 
                                    compute=True, 
                                    mode="w",
                                    zarr_version=3,
                                    encoding={"VH": {"compressors":zarr.codecs.BloscCodec(), 
                                                    "shards": (57,10000,10000), 
                                                    "dtype":"int16"}},
                                    consolidated=True)

### Inspecting Zarr store

In [ ]:
ds = xr.open_zarr("empty_2.zarr", group="s1sig0", consolidated=False, chunks=None)
ds

In [ ]:
filtered = ds.sel(time="2024-02-01T16:50:55.000000000", x=slice(5100010, 5100070), y=slice(1799990, 1799930))

In [ ]:
filtered.load()